## IMPORTING LIBRARIES

In [20]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(
    { "figure.figsize": (6, 4) },
    style='ticks',
    color_codes=True,
    font_scale=0.8
)
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import PredictionErrorDisplay
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### USING SYSTEM GPU

In [ ]:
import torch

def test_pytorch_gpu():
    print("PyTorch version:", torch.__version__)
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("CUDA is available. Using GPU.")
    else:
        device = torch.device("cpu")
        print("CUDA is not available. Using CPU.")
    
    # Tensor operation to test GPU
    x = torch.randn((2, 2), device=device)
    y = torch.randn((2, 2), device=device)
    z = x + y
    print("x + y on device:", z, "\nDevice:", z.device)
    print("Done.")

if __name__ == "__main__":
    test_pytorch_gpu()


PyTorch version: 2.6.0+cu118
CUDA is available. Using GPU.
x + y on device: tensor([[-2.3795,  0.0612],
        [-1.0989, -0.0473]], device='cuda:0') 
Device: cuda:0
Done.


In [6]:
car_auto = pd.read_csv('adverts.csv')

##  1. Data Description and Pre-Processing 

In [8]:
car_auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402005 entries, 0 to 402004
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   public_reference       402005 non-null  int64  
 1   mileage                401878 non-null  float64
 2   reg_code               370148 non-null  object 
 3   standard_colour        396627 non-null  object 
 4   standard_make          402005 non-null  object 
 5   standard_model         402005 non-null  object 
 6   vehicle_condition      402005 non-null  object 
 7   year_of_registration   368694 non-null  float64
 8   price                  402005 non-null  int64  
 9   body_type              401168 non-null  object 
 10  crossover_car_and_van  402005 non-null  bool   
 11  fuel_type              401404 non-null  object 
dtypes: bool(1), float64(2), int64(2), object(7)
memory usage: 34.1+ MB


In [ ]:
car_auto.head(7)

In [ ]:
car_auto.describe(include='all')

In [12]:
car_auto.isna().sum()

public_reference             0
mileage                    127
reg_code                 31857
standard_colour           5378
standard_make                0
standard_model               0
vehicle_condition            0
year_of_registration     33311
price                        0
body_type                  837
crossover_car_and_van        0
fuel_type                  601
dtype: int64

### 1.2 PREPROCESSING PIPELINE

Creating a pipeline to identify and handle missing values, outliers, noise; encode categorical variables; scale features; partition data into training, validation and test sets.

1. Cap Outliers (Mileage) with quantile-based boundaries.

2. Impute Mileage by (standard_make, vehicle_condition) group median.

3. Fix Year

 For rows with year_of_registration < 1933, fill from reg_code group median.

If vehicle_condition == 'NEW' and missing, set to 2021.

Otherwise, brand-based median fallback, then global median.

4. Parse public_reference into (year_listed, month_listed, day_listed).

5. Impute Standard Colour by (standard_make, vehicle_condition) mode.

6. Impute Body Type & Fuel Type by standard_make mode.

7. Map categories in body_type, fuel_type, and standard_colour to consolidated groups.

8. Drop reg_code (after it’s been used in the year fix).

In [21]:


# ─────────────────────────────────────────────────────────────────
# A) CapOutliers
# ─────────────────────────────────────────────────────────────────
class CapOutliers(BaseEstimator, TransformerMixin):
    """
    Caps outliers in a specified numeric column based on quantiles.
    """
    def __init__(self, column, lower_quantile=0.05, upper_quantile=0.95, factor=1.5):
        self.column = column
        self.lower_quantile = lower_quantile
        self.upper_quantile = upper_quantile
        self.factor = factor
        self.lower_bound_ = None
        self.upper_bound_ = None

    def fit(self, X, y=None):
        Q1 = X[self.column].quantile(self.lower_quantile)
        Q3 = X[self.column].quantile(self.upper_quantile)
        IQR = Q3 - Q1
        self.lower_bound_ = Q1 - self.factor * IQR
        self.upper_bound_ = Q3 + self.factor * IQR
        return self

    def transform(self, X):
        X = X.copy()
        X[self.column] = X[self.column].clip(lower=self.lower_bound_, upper=self.upper_bound_)
        return X

# ─────────────────────────────────────────────────────────────────
# B) GroupMedianImputerMileage
# ─────────────────────────────────────────────────────────────────
class GroupMedianImputerMileage(BaseEstimator, TransformerMixin):
    """
    Replace missing values in 'mileage' with the median mileage grouped by
    ('standard_make', 'vehicle_condition'). Then fallback to global median.
    """
    def __init__(self):
        self.group_medians_ = None
        self.global_median_ = None

    def fit(self, X, y=None):
        X_temp = X.copy()
        self.group_medians_ = X_temp.groupby(
            ['standard_make', 'vehicle_condition']
        )['mileage'].median()
        self.global_median_ = X_temp['mileage'].median()
        return self

    def transform(self, X):
        X = X.copy()

        def fill_mileage(row):
            if pd.isna(row['mileage']):
                return self.group_medians_.get(
                    (row['standard_make'], row['vehicle_condition']),
                    self.global_median_  # fallback
                )
            return row['mileage']

        X['mileage'] = X.apply(fill_mileage, axis=1)
        return X

# ─────────────────────────────────────────────────────────────────
# C) YearRegistrationFixer
#    Uses 'reg_code' for out-of-range years, then drops 'reg_code' later
# ─────────────────────────────────────────────────────────────────
class YearRegistrationFixer(BaseEstimator, TransformerMixin):
    """
    - For year_of_registration < 1933 => fill from reg_code median if available
    - If vehicle_condition == NEW and missing => 2021
    - Else fill missing with brand-based median => global fallback
    """
    def __init__(self):
        self.regcode_medians_ = None
        self.brand_medians_ = None
        self.global_median_ = None

    def fit(self, X, y=None):
        X_temp = X.copy()

        # 1) regcode-based median for out-of-range
        if 'reg_code' in X_temp.columns:
            self.regcode_medians_ = X_temp.groupby('reg_code')['year_of_registration'].median()

        # 2) brand-based median for missing
        self.brand_medians_ = X_temp.groupby('standard_make')['year_of_registration'].median()
        self.global_median_ = X_temp['year_of_registration'].median()

        return self

    def transform(self, X):
        X = X.copy()

        def fix_out_of_range(row):
            y = row['year_of_registration']
            if pd.notna(y) and y < 1933:
                # Use regcode median if available
                if self.regcode_medians_ is not None:
                    return self.regcode_medians_.get(row['reg_code'], y)
            return y

        X['year_of_registration'] = X.apply(fix_out_of_range, axis=1)

        # Next: vehicle_condition == NEW & missing => 2021
        mask_new_and_nan = (X['vehicle_condition'] == 'NEW') & (X['year_of_registration'].isnull())
        X.loc[mask_new_and_nan, 'year_of_registration'] = 2021

        # Then fill remaining missing with brand median => fallback global
        def fill_brand_median(row):
            y = row['year_of_registration']
            if pd.isna(y):
                return self.brand_medians_.get(row['standard_make'], np.nan)
            return y

        X['year_of_registration'] = X.apply(fill_brand_median, axis=1)
        X['year_of_registration'] = X['year_of_registration'].fillna(self.global_median_)

        # Convert to int
        X['year_of_registration'] = X['year_of_registration'].astype(int, errors='ignore')

        return X

# ─────────────────────────────────────────────────────────────────
# D) ParsePublicReference
# ─────────────────────────────────────────────────────────────────
class ParsePublicReference(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['public_reference'] = X['public_reference'].astype(str)

        X['year_listed'] = X['public_reference'].str[:4].astype(int)
        X['month_listed'] = X['public_reference'].str[4:6].astype(int)
        X['day_listed'] = X['public_reference'].str[6:8].astype(int)

        # We'll drop public_reference in a separate step if we want
        return X

# ─────────────────────────────────────────────────────────────────
# E) StandardColourImputer
# ─────────────────────────────────────────────────────────────────
class StandardColourImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        def impute_with_mode(group):
            mode_val = group['standard_colour'].mode()
            if not mode_val.empty:
                return group['standard_colour'].fillna(mode_val[0])
            else:
                return group['standard_colour'].fillna('Unknown')

        # Group by (make, condition) for color
        X['standard_colour'] = (
            X.groupby(['standard_make', 'vehicle_condition'], group_keys=False)
             .apply(impute_with_mode)
        )
        # Final fallback
        X['standard_colour'] = X['standard_colour'].fillna('Unknown')
        return X

# ─────────────────────────────────────────────────────────────────
# F) BodyFuelImputer
# ─────────────────────────────────────────────────────────────────
class BodyFuelImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Body type by standard_make
        def fill_body_mode(group):
            mode_val = group['body_type'].mode()
            return group['body_type'].fillna(mode_val[0] if not mode_val.empty else 'unknown')

        X['body_type'] = (
            X.groupby('standard_make', group_keys=False)
             .apply(fill_body_mode)
        )

        # Fuel type by standard_make
        def fill_fuel_mode(group):
            mode_val = group['fuel_type'].mode()
            return group['fuel_type'].fillna(mode_val[0] if not mode_val.empty else 'Unknown')

        X['fuel_type'] = (
            X.groupby('standard_make', group_keys=False)
             .apply(fill_fuel_mode)
        )

        return X

# ─────────────────────────────────────────────────────────────────
# G) MapBodyTypeFuelColour
# ─────────────────────────────────────────────────────────────────
class MapBodyTypeFuelColour(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.body_type_mapping = {
            'Hatchback': 'Hatchback',
            'SUV': 'SUV',
            'Saloon': 'Saloon',
            'Estate': 'Estate',
            'Coupe': 'Coupe',
            'Convertible': 'Convertible',
            'MPV': 'MPV',
            'Pickup': 'Utility Vehicles',
            'Combi Van': 'Utility Vehicles',
            'Panel Van': 'Utility Vehicles',
            'Window Van': 'Utility Vehicles',
            'Camper': 'Utility Vehicles',
            'Limousine': 'Luxury/Passenger Vehicles',
            'Minibus': 'Luxury/Passenger Vehicles',
            'Chassis Cab': 'Utility Vehicles',
            'Car Derived Van': 'Utility Vehicles',
            'unknown': 'Utility Vehicles'
        }

        self.fuel_type_mapping = {
            'Petrol': 'Petrol',
            'Diesel': 'Diesel',
            'Electric': 'Electric',
            'Petrol Hybrid': 'Hybrid',
            'Diesel Hybrid': 'Hybrid',
            'Petrol Plug-in Hybrid': 'Hybrid',
            'Diesel Plug-in Hybrid': 'Hybrid',
            'Natural Gas': 'Bi/Gas',
            'Bi Fuel': 'Bi/Gas'
        }

        self.color_mapping = {
            'Bronze': 'Metallic',
            'Gold': 'Metallic',
            'Beige': 'Metallic',
            'Pink': 'Red/Purple',
            'Purple': 'Red/Purple',
            'Magenta': 'Red/Purple',
            'Burgundy': 'Red/Purple',
            'Multicolour': 'Custom',
            'Turquoise': 'Custom',
            'Yellow': 'Yellow',
            'Brown': 'Brown',
            'Navy': 'Blue',
            'Maroon': 'Red/Purple',
            'Unknown': 'Unknown',
            'Indigo': 'Blue'
        }

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Body types
        X['body_type'] = X['body_type'].map(self.body_type_mapping).fillna('Utility Vehicles')

        # Fuel
        X['fuel_type'] = X['fuel_type'].map(self.fuel_type_mapping).fillna('Unknown')

        # Colour
        X['standard_colour'] = X['standard_colour'].replace(self.color_mapping)

        return X

# ─────────────────────────────────────────────────────────────────
# H) DropColumns
#    We'll drop 'public_reference' and 'reg_code' (the latter after it's used)
# ─────────────────────────────────────────────────────────────────
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, cols_to_drop):
        self.cols_to_drop = cols_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X.drop(columns=self.cols_to_drop, inplace=True, errors='ignore')
        return X


In [22]:
def create_cleaning_pipeline():
    """
    Creates a pipeline that:
      1) Caps outliers in 'mileage'
      2) Imputes 'mileage' by (make, condition) median
      3) Fixes 'year_of_registration' using reg_code for <1933, then brand-based median
      4) Parses 'public_reference' => year_listed, month_listed, day_listed
      5) Imputes standard_colour by (make, condition) mode
      6) Imputes body_type, fuel_type by make mode
      7) Maps categories for body_type, fuel_type, standard_colour
      8) Drops 'reg_code' and 'public_reference'
    """
    return Pipeline([
        ("cap_outliers", CapOutliers(column="mileage", lower_quantile=0.05, upper_quantile=0.95)),
        ("impute_mileage", GroupMedianImputerMileage()),
        ("fix_year", YearRegistrationFixer()),
        ("parse_pubref", ParsePublicReference()),
        ("impute_colour", StandardColourImputer()),
        ("impute_bodyfuel", BodyFuelImputer()),
        ("map_bfc", MapBodyTypeFuelColour()),
        # Finally drop columns we don't need in the final dataset:
        ("drop_unused", DropColumns(cols_to_drop=["reg_code", "public_reference"]))
    ])


In [24]:
# Suppose your raw DataFrame is car_auto
pipeline = create_cleaning_pipeline()

# Fit-transform
car_auto_cl = pipeline.fit_transform(car_auto)

print("Shape after cleaning:", car_auto_cl.shape)
print(car_auto_cl.info())
car_auto_cl.head(10)


Shape after cleaning: (402005, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402005 entries, 0 to 402004
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   mileage                402005 non-null  float64
 1   standard_colour        402005 non-null  object 
 2   standard_make          402005 non-null  object 
 3   standard_model         402005 non-null  object 
 4   vehicle_condition      402005 non-null  object 
 5   year_of_registration   402005 non-null  int64  
 6   price                  402005 non-null  int64  
 7   body_type              402005 non-null  object 
 8   crossover_car_and_van  402005 non-null  bool   
 9   fuel_type              402005 non-null  object 
 10  year_listed            402005 non-null  int64  
 11  month_listed           402005 non-null  int64  
 12  day_listed             402005 non-null  int64  
dtypes: bool(1), float64(1), int64(5), object(6)
memory usa

,mileage,standard_colour,standard_make,standard_model,vehicle_condition,year_of_registration,price,body_type,crossover_car_and_van,fuel_type,year_listed,month_listed,day_listed
0,0.0,Grey,Volvo,XC90,NEW,2021,73970,SUV,False,Hybrid,2020,6,3
1,108230.0,Blue,Jaguar,XF,USED,2011,7000,Saloon,False,Diesel,2020,7,2
2,7800.0,Grey,SKODA,Yeti,USED,2017,14000,SUV,False,Petrol,2020,7,2
3,45000.0,Brown,Vauxhall,Mokka,USED,2016,7995,Hatchback,False,Diesel,2020,7,8
4,64000.0,Grey,Land Rover,Range Rover Sport,USED,2015,26995,SUV,False,Diesel,2020,7,16
5,16000.0,Blue,Audi,S5,USED,2017,29000,Convertible,False,Petrol,2020,9,30
6,24075.0,Red,Vauxhall,Viva,USED,2017,5861,Hatchback,False,Petrol,2020,7,8
7,99000.0,Metallic,Land Rover,Range Rover,USED,2013,24475,SUV,False,Diesel,2020,9,24
8,111236.0,Black,Mercedes-Benz,S Class,USED,2008,6995,Luxury/Passenger Vehicles,False,Diesel,2020,10,1
9,9500.0,White,Land Rover,Range Rover,USED,2019,79995,SUV,False,Diesel,2020,6,23


In [25]:
car_auto_cl.to_csv('adverts_cl.csv', index=False)